# Movie recommendation using MovieLens data

Importing required packages

In [121]:
import pandas as pd
import graphlab
from sklearn.model_selection import train_test_split,KFold
from matplotlib import pyplot as plt
import numpy as np
import nltk

## Data exploration

Importing datasets from csv file

In [2]:
links = pd.read_csv("links.csv")
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")
tags = pd.read_csv("tags.csv")

Exploring Links dataset

In [3]:
#checking the head of the links dataset (at movie ID level)
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
#checking the shape of the links dataset
links_shape = links.shape
print("Shape of links dataset = {}".format(links_shape))

Shape of links dataset = (9125, 3)


In [5]:
#number of unique movie Ids in the links dataset
uni_movieIds_links = links["movieId"].nunique()
print("No of unique movieIds in links dataset = {}".format(uni_movieIds_links))

No of unique movieIds in links dataset = 9125


In [6]:
#checking if a column has null values 
links.isnull().any()

movieId    False
imdbId     False
tmdbId      True
dtype: bool

In [7]:
#looking at rows with missing tmdbid
links[links["tmdbId"].isnull()].head()

,movieId,imdbId,tmdbId
607,720,118114,NaN
608,721,114103,NaN
640,769,116992,NaN
910,1133,111357,NaN
2282,2851,81454,NaN


In [8]:
#number of movies from which tmdbid is missing (13)
missing_tmdbId = links.loc[links["tmdbId"].isnull(), "movieId"].nunique()
print("{} movies have missing tmdbId".format(missing_tmdbId))

13 movies have missing tmdbId


In [9]:
#Replacing the missing TmdbId with Zero
links.loc[(links.tmdbId.isnull()) , "tmdbId"] = 0

Exploring Movies Dataset

In [10]:
#checking the head of the movies dataset (at movie ID level)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
#checking the shape of the movies dataset
movies_shape = movies.shape
print("Shape of movies dataset = {}".format(movies_shape))

Shape of movies dataset = (9125, 3)


In [12]:
#Data is unique at movie Id level
no_movies = movies["movieId"].nunique()
print("No of unique movies in movies dataset = {}".format(no_movies))

No of unique movies in movies dataset = 9125


In [13]:
#checking for mising values 
movies.isnull().any()

movieId    False
title      False
genres     False
dtype: bool

In [14]:
#checking if the genres have not been listed
no_genre = movies.loc[movies.genres == "(no genres listed)"].movieId.count()
print("{} movies dont have genres listed".format(no_genre) )

18 movies dont have genres listed


In [15]:
#creating a new dataset with genres split into different columns
movies2 = pd.DataFrame(movies.genres.str.split('|').tolist(), columns = ["genres1","genres2","genres3","genres4","genres5","genres6","genres7","genres8","genres9","genres10"])

In [16]:
#concatenating the two datasets to create the final movies dataset
movies_new = pd.concat([movies,movies2],axis = 1)
movies_new.head()

,movieId,title,genres,genres1,genres2,genres3,genres4,genres5,genres6,genres7,genres8,genres9,genres10
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure,Children,Fantasy,None,None,None,None,None,None,None
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy,Romance,None,None,None,None,None,None,None,None
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy,Drama,Romance,None,None,None,None,None,None,None
4,5,Father of the Bride Part II (1995),Comedy,Comedy,None,None,None,None,None,None,None,None,None


In [17]:
#creating the year of release dataset from the title column
movies_new["Year_release"] = movies_new['title'].str.extract('(\d\d\d\d)', expand=True)
movies_new.head()

,movieId,title,genres,genres1,genres2,genres3,genres4,genres5,genres6,genres7,genres8,genres9,genres10,Year_release
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure,Children,Fantasy,None,None,None,None,None,None,None,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy,Romance,None,None,None,None,None,None,None,None,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy,Drama,Romance,None,None,None,None,None,None,None,1995
4,5,Father of the Bride Part II (1995),Comedy,Comedy,None,None,None,None,None,None,None,None,None,1995


In [18]:
movies_new.isnull().any()

movieId         False
title           False
genres          False
genres1         False
genres2          True
genres3          True
genres4          True
genres5          True
genres6          True
genres7          True
genres8          True
genres9          True
genres10         True
Year_release     True
dtype: bool

In [19]:
#No of movies with missing year of release 
missing_year = movies_new.loc[movies_new['Year_release'].isnull(),'movieId'].nunique()
print("{} movies have missing year of release".format(missing_year))

4 movies have missing year of release


In [20]:
#Adding the missing Year_release for 4 movies manually
movies_new.loc[9017,"Year_release"] = "2015"
movies_new.loc[9063,"Year_release"] = "2016"
movies_new.loc[9118,"Year_release"] = "2016"
movies_new.loc[9124,"Year_release"] = "2014"

In [21]:
#merging the the links and movies datasets as both are on movie ID level
movies_links = pd.merge(movies_new,links, on = "movieId")
movies_links.head()
movies_links.shape
movies_links_shape = movies_links.shape
print("Shape of movies_links dataset = {}".format(movies_links_shape))

Shape of movies_links dataset = (9125, 16)


In [22]:
movies_links.isnull().any()

movieId         False
title           False
genres          False
genres1         False
genres2          True
genres3          True
genres4          True
genres5          True
genres6          True
genres7          True
genres8          True
genres9          True
genres10         True
Year_release    False
imdbId          False
tmdbId          False
dtype: bool

In [22]:
#storing movie and links data as Sframe
movies_links.to_csv("movies_links.csv")
movie_metadata = graphlab.SFrame.read_csv('movies_links.csv' )

This non-commercial license of GraphLab Create for academic use is assigned to dh2863@columbia.edu and will expire on November 02, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\hp\AppData\Local\Temp\graphlab_server_1509991816.log.0


Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\movies_links.csv

Parsing completed. Parsed 100 lines in 0.086417 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,str,str,str,str,str,str,str,str,str,str,str,str,long,long,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\movies_links.csv

Parsing completed. Parsed 9125 lines in 0.053039 secs.

Exploring Ratings Data

In [23]:
#checking the head of the ratings data (unique at userID, movieID, timestamp level)
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [24]:
#checking the shape of the ratings data
ratings_shape = ratings.shape
print("{} is the shape of ratings data".format(ratings_shape))

(100004, 4) is the shape of ratings data


In [28]:
#No missing entries in ratings data
ratings.isnull().any()

userId       False
movieId      False
rating       False
timestamp    False
dtype: bool

In [29]:
#No of unique users 671
m = ratings.nunique()
uni_user = m[0]
uni_movies = m[1]
uni_ratings = m[2]
print("No of unique users who have rated a movie = {} ".format(uni_user))
print("No of unique movies that have been rated = {} ".format(uni_movies))
print("No of unique values of ratings = {} ".format(uni_ratings))

userId         671
movieId       9066
rating          10
timestamp    78141
dtype: int64
No of unique users who have rated a movie = 671 
No of unique movies that have been rated = 9066 
No of unique values of ratings = 10 


In [30]:
#unique values of ratings 
k = ratings['rating'].unique()
k = np.sort(k, kind= "quicksort")
print("Values of ratings =  {}".format(k))

Values of ratings =  [ 0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5. ]


In [31]:
#overall distribution of ratings 
ratings.groupby('rating').userId.count()

rating
0.5     1101
1.0     3326
1.5     1687
2.0     7271
2.5     4449
3.0    20064
3.5    10538
4.0    28750
4.5     7723
5.0    15095
Name: userId, dtype: int64

In [32]:
#Every customer has rated a movie only once
ratings.groupby(['userId','movieId']).userId.count().value_counts()

1    100004
Name: userId, dtype: int64

In [33]:
#converting time since epoch to datetime 
ratings["timestamp"] = pd.to_datetime(ratings['timestamp'], unit='ms')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1970-01-15 14:12:39.144
1,1,1029,3.0,1970-01-15 14:12:39.179
2,1,1061,3.0,1970-01-15 14:12:39.182
3,1,1129,2.0,1970-01-15 14:12:39.185
4,1,1172,4.0,1970-01-15 14:12:39.205


Explaoring Tags data

In [34]:
#checking tags data 
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


In [35]:
#converting time since epoch to datetime 
tags["timestamp"] = pd.to_datetime(tags['timestamp'], unit='ms')
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1970-01-14 04:15:37.770
1,15,1955,dentist,1970-01-14 19:30:35.061
2,15,7478,Cambodia,1970-01-14 13:09:20.997
3,15,32892,Russian,1970-01-14 13:10:26.366
4,15,34162,forgettable,1970-01-14 05:03:11.765


In [36]:
#shape of tags data
tags_shape = tags.shape
print("Shape of tags dataset = {}".format(tags_shape))

Shape of tags dataset = (1296, 4)


In [37]:
#No of unique values in tags dataset
m = tags.nunique()
uni_user = m[0]
uni_movies = m[1]
uni_tags = m[2]
print("No of unique users who have tagged a movie = {} ".format(uni_user))
print("No of unique movies that have been tagged = {} ".format(uni_movies))
print("No of unique values of tags = {} ".format(uni_tags))

No of unique users who have tagged a movie = 61 
No of unique movies that have been tagged = 689 
No of unique values of tags = 582 


In [38]:
#checking Null values in the tags dataset
tags.isnull().any()

userId       False
movieId      False
tag          False
timestamp    False
dtype: bool

In [39]:
#looking at data with more than 5 tags for a movie
df = tags.groupby(['userId','movieId']).count()
df.loc[df['tag'] > 5].head()

tag  timestamp
userId movieId                
138    48780      6          6
       79132      7          7
       109487    12         12
212    64957      8          8
       66097      7          7

In [40]:
#creating combined tag for a particular user movie combination
tags_new =tags.groupby(['userId','movieId'], as_index=False).sum()

In [41]:
tags_new.head()

,userId,movieId,tag
0,15,339,sandra 'boring' bullock
1,15,1955,dentist
2,15,7478,Cambodia
3,15,32892,Russian
4,15,34162,forgettable


In [42]:
#New shape of tags dataset
tags_shape = tags_new.shape
print("shape of tags dataset = {}".format(tags_shape))

shape of tags dataset = (772, 3)


In [43]:
#merging the ratings and tags data : information at user level
ratings_tags = pd.merge(ratings,tags_new[['userId','movieId','tag']],how = 'left', left_on = ['userId','movieId'], right_on = ['userId','movieId'])

In [44]:
#shape of the new ratings _tags dataset
ratings_tags_shape = ratings_tags.shape
print("shape of ratings_tags dataset = {}".format(ratings_tags_shape))

shape of ratings_tags dataset = (100004, 5)


In [45]:
#checking data for a particular entry
ratings_tags.loc[(ratings_tags.userId == 15) & (ratings_tags.movieId == 339)]

,userId,movieId,rating,timestamp,tag
1048,15,339,2.5,1970-01-13 23:49:36.622,sandra 'boring' bullock


In [46]:
#storing ratings and tags data as Sframe
ratings_tags.to_csv("ratings_tags.csv")
user_metadata = graphlab.SFrame.read_csv('ratings_tags.csv' )

Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\ratings_tags.csv

Parsing completed. Parsed 100 lines in 0.1351 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,long,long,float,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\ratings_tags.csv

Parsing completed. Parsed 100004 lines in 0.148604 secs.

In [47]:
#Including the movie information to the ratings_tags data
all_data = pd.merge(ratings_tags, movies_links,how = 'left', left_on = ['movieId'], right_on = ['movieId'])

In [48]:
#checking the head of the combined data
all_data.head()

,userId,movieId,rating,timestamp,tag,title,genres,genres1,genres2,genres3,genres4,genres5,genres6,genres7,genres8,genres9,genres10,Year_release,imdbId,tmdbId
0,1,31,2.5,1970-01-15 14:12:39.144,NaN,Dangerous Minds (1995),Drama,Drama,None,None,None,None,None,None,None,None,None,1995,112792,9909.0
1,1,1029,3.0,1970-01-15 14:12:39.179,NaN,Dumbo (1941),Animation|Children|Drama|Musical,Animation,Children,Drama,Musical,None,None,None,None,None,None,1941,33563,11360.0
2,1,1061,3.0,1970-01-15 14:12:39.182,NaN,Sleepers (1996),Thriller,Thriller,None,None,None,None,None,None,None,None,None,1996,117665,819.0
3,1,1129,2.0,1970-01-15 14:12:39.185,NaN,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,Action,Adventure,Sci-Fi,Thriller,None,None,None,None,None,None,1981,82340,1103.0
4,1,1172,4.0,1970-01-15 14:12:39.205,NaN,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,Drama,None,None,None,None,None,None,None,None,None,1989,95765,11216.0


In [49]:
#shape of the combined data
all_data_shape = all_data.shape
print("shape of combined dataset = {}".format(all_data_shape))

shape of combined dataset = (100004, 20)


In [50]:
#converting the tags column to string type
all_data['tag'] = all_data['tag'].astype(str)

In [51]:
#checking data type of all columns 
all_data.dtypes

userId                   int64
movieId                  int64
rating                 float64
timestamp       datetime64[ns]
tag                     object
title                   object
genres                  object
genres1                 object
genres2                 object
genres3                 object
genres4                 object
genres5                 object
genres6                 object
genres7                 object
genres8                 object
genres9                 object
genres10                object
Year_release            object
imdbId                   int64
tmdbId                 float64
dtype: object

# Model based recommendation with Graphlabs

In [59]:
#splitting the data into test and train 80/20
train_data, test_data  = train_test_split(all_data , test_size=0.2, random_state=42)

In [60]:
#shape of train data
train_shape = train_data.shape
print("shape of train dataset = {}".format(train_shape))

shape of train dataset = (80003, 20)


In [61]:
#No of unique values in train data
m = train_data.nunique()
uni_user = m[0]
uni_movies = m[1]
uni_ratings = m[2]
print("No of unique users who have rated a movie = {} ".format(uni_user))
print("No of unique movies that have been rated = {} ".format(uni_movies))
print("No of unique values of ratings = {} ".format(uni_ratings))

No of unique users who have rated a movie = 671 
No of unique movies that have been rated = 8399 
No of unique values of ratings = 10 


In [62]:
#shape of test dataset
test_shape = test_data.shape
print("shape of train dataset = {}".format(test_shape))

shape of train dataset = (20001, 20)


In [63]:
#No of unique values in test data
m = test_data.nunique()
uni_user = m[0]
uni_movies = m[1]
uni_ratings = m[2]
print("No of unique users who have rated a movie = {} ".format(uni_user))
print("No of unique movies that have been rated = {} ".format(uni_movies))
print("No of unique values of ratings = {} ".format(uni_ratings))

No of unique users who have rated a movie = 671 
No of unique movies that have been rated = 4901 
No of unique values of ratings = 10 


In [68]:
#Keeping a copy of the test and train dataframes to use for sampling later
trainData = train_data
testData = test_data
#verifying
print(trainData.shape)
print(testData.shape)

(80003, 20)
(20001, 20)


In [69]:
#exporting the test and train dataset to csv 
train_data.to_csv('train_data.csv')
test_data.to_csv('test_data.csv')

In [71]:
#converting the test and train dataset to Sframe
hints = [int,int ,int ,float,str, str, str, str, str, str,str,str,str,str,str,str, str,str,float, int, float]
train = graphlab.SFrame.read_csv('train_data.csv',column_type_hints = hints )
test = graphlab.SFrame.read_csv('test_data.csv',column_type_hints = hints )

Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\train_data.csv

Parsing completed. Parsed 80003 lines in 0.425811 secs.

Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\test_data.csv

Parsing completed. Parsed 20001 lines in 0.111292 secs.

Using the follwoing three approaches to solve the regression.
* Adaptive Gradient Stochastic Gradient Descent
* Stochastic Gradient Descent
* Alternating Least Squares

In [72]:
#building a factorization_recommender model using the graphLab package with 4 latent factors
#Using adaptive gradient stochastic descent 
m1 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',solver= "adagrad" , user_data = user_metadata, item_data = movie_metadata, num_factors=4)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 0.995481s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0138882

Final training RMSE: 0.117206

In [73]:
#Using stochastic gradient descent 
m2 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId', user_data = user_metadata, item_data = movie_metadata ,solver= "sgd" , num_factors=4)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 1.61617s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.905065

Final training RMSE: 0.951349

In [74]:
#Using Implicit Alternating Least Squares 
m3 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',solver= "als" ,user_data = user_metadata, item_data = movie_metadata , num_factors=4)


Recsys training: model = factorization_recommender

Warning: This solver currently does not support side features. Proceeding with training without side features.

Preparing data set.

Data has 80003 observations with 671 users and 8399 items.

Data prepared in: 0.782118s

...

| 48      | 4.34s        | 0.687974               |

| 49      | 4.43s        | 0.687716               |

| FINAL   | 4.43s        | 0.687716               |

+---------+--------------+------------------------+

Optimization Complete: Iteration limit reached.

Comparing the performance of the three models on the test data, to finalize the approach

In [60]:
 model_comp = graphlab.compare(test, [m1,m2, m3])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |  0.00298062593145 | 4.54208768326e-05 |
|   2    |  0.00149031296572 | 4.54208768326e-05 |
|   3    | 0.000993541977149 | 4.54208768326e-05 |
|   4    |  0.00111773472429 | 0.000160060335734 |
|   5    | 0.000894187779434 | 0.000160060335734 |
|   6    | 0.000745156482861 | 0.000160060335734 |
|   7    | 0.000638705556738 | 0.000160060335734 |
|   8    | 0.000558867362146 | 0.000160060335734 |
|   9    | 0.000496770988574 | 0.000160060335734 |
|   10   | 0.000596125186289 | 0.000238497860246 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+----

In [75]:
m1.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+----------------+
 | movieId | count |      rmse      |
 +---------+-------+----------------+
 |   7899  |   2   | 0.612873853726 |
 |   3143  |   1   | 0.429013924694 |
 |   5684  |   1   | 1.52113013713  |
 |   2779  |   3   | 0.933228796413 |
 |   3988  |   4   | 1.09241422095  |
 |   2847  |   1   | 0.299188140434 |
 |  64614  |   9   | 0.700860623345 |
 |   2925  |   2   | 0.569762639691 |
 |   2871  |   5   | 1.15031923203  |
 |   3913  |   1   | 0.818596055241 |
 +---------+-------+----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |   34  | 0.84911

In [62]:
m2.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+-----------------+
 | movieId | count |       rmse      |
 +---------+-------+-----------------+
 |   7899  |   2   |  0.522340933412 |
 |   3143  |   1   |  0.205963887482 |
 |   5684  |   1   |  2.06886128989  |
 |   2779  |   3   |  0.645115488303 |
 |   3988  |   4   |   1.022573678   |
 |   2847  |   1   |  0.107805869628 |
 |  64614  |   9   |  0.693141330215 |
 |   2925  |   2   |  0.475602542774 |
 |   2871  |   5   |  0.938756426007 |
 |   3913  |   1   | 0.0705656712917 |
 +---------+-------+-----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |  

In [76]:
m3.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+----------------+
 | movieId | count |      rmse      |
 +---------+-------+----------------+
 |   7899  |   2   | 0.408983244419 |
 |   3143  |   1   | 0.458251565566 |
 |   5684  |   1   | 1.54174843443  |
 |   2779  |   3   |  1.2046916373  |
 |   3988  |   4   | 0.813569821433 |
 |   2847  |   1   | 0.59845843924  |
 |  64614  |   9   | 0.761183903438 |
 |   2925  |   2   | 1.44959885942  |
 |   2871  |   5   | 0.947890479543 |
 |   3913  |   1   | 0.199668466916 |
 +---------+-------+----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |   34  | 0.70633

ADA model performance for different value of max iterations

In [77]:
#25 iterations
ada_m1 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',solver= "adagrad",user_data = user_metadata, item_data = movie_metadata  ,max_iterations=25,num_factors=4)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 0.957236s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0331871

Final training RMSE: 0.181902

In [78]:
#50 iterations
ada_m2 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',user_data = user_metadata, item_data = movie_metadata,solver= "adagrad" ,max_iterations=50,num_factors=4)

Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 0.973228s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0119623

Final training RMSE: 0.108613

In [80]:
ada_m2['coefficients']

{'intercept': 3.5417484344337096, 'movieId': Columns:
 	movieId	int
 	linear_terms	float
 	factors	array
 
 Rows: 9125
 
 Data:
 +---------+--------------------+-------------------------------+
 | movieId |    linear_terms    |            factors            |
 +---------+--------------------+-------------------------------+
 |    1    |   0.102404944599   | [-0.312023937702, -0.05141... |
 |    2    |  0.0703761801124   | [-0.153851255774, -0.06863... |
 |    3    |  -0.138090670109   | [0.439986884594, 0.0623183... |
 |    4    |  -0.281930148602   | [0.258144617081, 0.2196142... |
 |    5    |   0.185211256146   | [0.0651989132166, -0.39491... |
 |    6    |   0.118924714625   | [0.330348193645, 0.2913644... |
 |    7    |  -0.0723969265819  | [-0.434919685125, -0.43319... |
 |    8    | -0.000643874052912 | [0.58164781332, -0.1067694... |
 |    9    |  0.0380612388253   | [0.579683661461, 0.2395905... |
 |    10   |  0.0249488372356   | [-0.0587613917887, 0.20702... |
 +---------+--

Creating a visualization of the Model to helps explore and qualitatively evaluate the recommendations made by the model

In [82]:
view = ada_m2.views.explore(train)
view.show()

View object

URI: 		http://localhost:32212/view/933d700b-f791-4e95-bd29-c55202ebf9ad
HTML: 		
<gl-recommender-explore
    uri="http://localhost:32212/view/131ef920-7a1e-4191-80f2-52358120876d"
    api_key=""
/>
        

In [69]:
 ada_model_iter = graphlab.compare(test, [ada_m1, ada_m2])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    | 0.000745156482861 | 8.27951647624e-05 |
|   3    |  0.00149031296572 | 0.000311061068202 |
|   4    |  0.00111773472429 | 0.000311061068202 |
|   5    | 0.000894187779434 | 0.000311061068202 |
|   6    | 0.000993541977149 | 0.000476651397726 |
|   7    |  0.00106450926123 | 0.000497641721187 |
|   8    |  0.00111773472429 | 0.000503555661527 |
|   9    | 0.000993541977149 | 0.000503555661527 |
|   10   |  0.00104321907601 | 0.000506693162508 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+----

In [83]:
ada_m1.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+----------------+
 | movieId | count |      rmse      |
 +---------+-------+----------------+
 |   7899  |   2   | 0.191592648828 |
 |   3143  |   1   | 0.234725949104 |
 |   5684  |   1   | 1.59879566772  |
 |   2779  |   3   | 1.52303128722  |
 |   3988  |   4   | 0.971305527676 |
 |   2847  |   1   | 0.681097631637 |
 |  64614  |   9   | 0.657623209602 |
 |   2925  |   2   | 0.412099789294 |
 |   2871  |   5   | 1.34029917568  |
 |   3913  |   1   | 0.25831775292  |
 +---------+-------+----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |   34  | 0.74105

In [84]:
ada_m2.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+----------------+
 | movieId | count |      rmse      |
 +---------+-------+----------------+
 |   7899  |   2   | 0.45693048359  |
 |   3143  |   1   | 0.740465724151 |
 |   5684  |   1   | 1.43961103432  |
 |   2779  |   3   | 1.07914969234  |
 |   3988  |   4   | 1.15282939067  |
 |   2847  |   1   | 0.488297998373 |
 |  64614  |   9   | 0.772381498031 |
 |   2925  |   2   | 0.143766005088 |
 |   2871  |   5   | 0.823893140134 |
 |   3913  |   1   | 0.418661601506 |
 +---------+-------+----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |   34  | 0.96778

In [85]:
#use ada_m2 to predict ratings 
ada_m2.recommend(users=range(1,11),k=1000)

userId,movieId,score,rank
1,94466,3.88882499219,1
1,128620,3.87071913363,2
1,152081,3.73122478996,3
1,68269,3.63943767526,4
1,134853,3.62677704103,5
1,67504,3.6223518746,6
1,115713,3.61247356146,7
1,79824,3.5993067426,8
1,112852,3.59454272627,9
1,8264,3.57194275994,10


Model performance for different values of L2 regularization

In [86]:
ada_R1 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',user_data = user_metadata, item_data = movie_metadata,solver= "adagrad" ,regularization=1e-10,num_factors=4)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 0.979231s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.00712102

Final training RMSE: 0.0843742

In [87]:
ada_R2 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',user_data = user_metadata, item_data = movie_metadata,solver= "adagrad" ,regularization=1e-8,num_factors=4)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 1.84784s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0096554

Final training RMSE: 0.0974414

In [88]:
ada_R3 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',user_data = user_metadata, item_data = movie_metadata,solver= "adagrad" ,regularization=1e-6,num_factors=4)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 0.98176s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0261211

Final training RMSE: 0.120451

In [89]:
 ada_model_reg = graphlab.compare(test, [ada_R1, ada_R2 ,ada_R3])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |  0.00149031296572 | 4.51609989613e-06 |
|   2    | 0.000745156482861 | 4.51609989613e-06 |
|   3    |  0.00149031296572 | 4.07677008936e-05 |
|   4    |  0.00149031296572 | 4.39052018741e-05 |
|   5    |  0.00119225037258 | 4.39052018741e-05 |
|   6    |  0.00124192747144 | 4.82884753027e-05 |
|   7    |  0.00106450926123 | 4.82884753027e-05 |
|   8    | 0.000931445603577 | 4.82884753027e-05 |
|   9    | 0.000993541977149 |  7.751029816e-05  |
|   10   | 0.000894187779434 |  7.751029816e-05  |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--

In [77]:
ada_R1.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+----------------+
 | movieId | count |      rmse      |
 +---------+-------+----------------+
 |   7899  |   2   | 0.258587158942 |
 |   3143  |   1   | 0.747133156962 |
 |   5684  |   1   | 1.44507143119  |
 |   2779  |   3   | 0.926701275881 |
 |   3988  |   4   | 0.93650724547  |
 |   2847  |   1   | 0.221362879229 |
 |  64614  |   9   | 0.759506007397 |
 |   2925  |   2   | 1.33987519159  |
 |   2871  |   5   | 1.39284310725  |
 |   3913  |   1   | 0.36085953014  |
 +---------+-------+----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |   34  | 0.91668

In [78]:
ada_R2.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+----------------+
 | movieId | count |      rmse      |
 +---------+-------+----------------+
 |   7899  |   2   | 0.490269126276 |
 |   3143  |   1   | 0.226173744892 |
 |   5684  |   1   | 1.57510679373  |
 |   2779  |   3   | 0.835510381516 |
 |   3988  |   4   | 0.819734445408 |
 |   2847  |   1   | 0.110462617344 |
 |  64614  |   9   | 0.796728211145 |
 |   2925  |   2   | 0.927408188074 |
 |   2871  |   5   | 0.898754714723 |
 |   3913  |   1   | 0.235724501857 |
 +---------+-------+----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |   34  | 0.78009

In [79]:
ada_R3.evaluate_rmse(test,target='rating')

{'rmse_by_item': Columns:
 	movieId	int
 	count	int
 	rmse	float
 
 Rows: 4901
 
 Data:
 +---------+-------+----------------+
 | movieId | count |      rmse      |
 +---------+-------+----------------+
 |   7899  |   2   | 0.591126191995 |
 |   3143  |   1   | 0.540916633661 |
 |   5684  |   1   | 1.39918696549  |
 |   2779  |   3   | 0.371713502663 |
 |   3988  |   4   | 1.02225046852  |
 |   2847  |   1   | 0.127944276775 |
 |  64614  |   9   | 0.675814668016 |
 |   2925  |   2   | 0.319005747806 |
 |   2871  |   5   | 0.803685322664 |
 |   3913  |   1   | 0.535754216071 |
 +---------+-------+----------------+
 [4901 rows x 3 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.,
 'rmse_by_user': Columns:
 	userId	int
 	count	int
 	rmse	float
 
 Rows: 671
 
 Data:
 +--------+-------+----------------+
 | userId | count |      rmse      |
 +--------+-------+----------------+
 |  118   |   34  | 0.81687

In [101]:
ada_NF1 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',user_data = user_metadata, item_data = movie_metadata,solver= "adagrad" ,regularization=1e-6,num_factors=2)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 0.952677s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0444807

Final training RMSE: 0.181077

In [100]:
ada_NF2 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',user_data = user_metadata, item_data = movie_metadata,solver= "adagrad" ,regularization=1e-6,num_factors=4)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 1.05579s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0229877

Final training RMSE: 0.0881678

In [99]:
ada_NF3 = graphlab.factorization_recommender.create(train, target='rating', user_id='userId', item_id='movieId',user_data = user_metadata, item_data = movie_metadata,solver= "adagrad" ,regularization=1e-6,num_factors=6)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

Data prepared in: 1.06178s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.0169506

Final training RMSE: 0.0711118

In [103]:
graphlab.recommender.util.compare_models(test, [ada_NF1,ada_NF2,ada_NF3], model_names=["ada_NF1","ada_NF2","ada_NF3"],
                                         user_sample=1.0, metric='auto',
                                         target='rating', exclude_known_for_precision_recall=True, 
                                         make_plot=True, verbose=False)

[{'precision_recall_by_user': Columns:
  	userId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 12078
  
  Data:
  +--------+--------+-----------+--------+-------+
  | userId | cutoff | precision | recall | count |
  +--------+--------+-----------+--------+-------+
  |   1    |   1    |    0.0    |  0.0   |   1   |
  |   1    |   2    |    0.0    |  0.0   |   1   |
  |   1    |   3    |    0.0    |  0.0   |   1   |
  |   1    |   4    |    0.0    |  0.0   |   1   |
  |   1    |   5    |    0.0    |  0.0   |   1   |
  |   1    |   6    |    0.0    |  0.0   |   1   |
  |   1    |   7    |    0.0    |  0.0   |   1   |
  |   1    |   8    |    0.0    |  0.0   |   1   |
  |   1    |   9    |    0.0    |  0.0   |   1   |
  |   1    |   10   |    0.0    |  0.0   |   1   |
  +--------+--------+-----------+--------+-------+
  [12078 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to print more ro

# Evaluating Model Performance For Different Sample Sizes

In [94]:
train_sample1 = trainData.sample(frac =0.25,replace =False)
train_sample2 = trainData.sample(frac =0.5,replace =False)
train_sample3 = trainData.sample(frac =0.75,replace =False)
print("Training sample-1 shape - {}".format(train_sample1.shape))
print("Training sample-2 shape - {}".format(train_sample2.shape))
print("Training sample-3 shape - {}".format(train_sample3.shape))

Training sample-1 shape - (20001, 20)
Training sample-2 shape - (40002, 20)
Training sample-3 shape - (60002, 20)


In [96]:
#converting data to csv for Sframe parsing
train_sample1.to_csv("train_data1.csv")
train_sample2.to_csv("train_data2.csv")
train_sample3.to_csv("train_data3.csv")

In [97]:
#converting the train datasets to Sframe
hints = [int,int ,int ,float,str, str, str, str, str, str,str,str,str,str,str,str, str,str,float, int, float]
train1 = graphlab.SFrame.read_csv('train_data1.csv',column_type_hints = hints )
train2 = graphlab.SFrame.read_csv('train_data2.csv',column_type_hints = hints )
train3 = graphlab.SFrame.read_csv('train_data3.csv',column_type_hints = hints )

Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\train_data1.csv

Parsing completed. Parsed 20001 lines in 0.159123 secs.

Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\train_data2.csv

Parsing completed. Parsed 40002 lines in 0.220673 secs.

Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\train_data3.csv

Parsing completed. Parsed 60002 lines in 0.300706 secs.

Building a factorization_recommender model using the graphLab package with 4 latent factors
Using adaptive gradient stochastic descent 

In [104]:
m1_t1 = graphlab.factorization_recommender.create(train1, target='rating', user_id='userId',
                                               item_id='movieId',solver= "adagrad" , user_data = user_metadata,
                                               item_data = movie_metadata, num_factors=4)
m1_t2 = graphlab.factorization_recommender.create(train2, target='rating', user_id='userId',
                                               item_id='movieId',solver= "adagrad" , user_data = user_metadata,
                                               item_data = movie_metadata, num_factors=4)
m1_t3 = graphlab.factorization_recommender.create(train3, target='rating', user_id='userId',
                                               item_id='movieId',solver= "adagrad" , user_data = user_metadata,
                                               item_data = movie_metadata, num_factors=4)


Recsys training: model = factorization_recommender

Preparing data set.

Data has 20001 observations with 671 users and 9125 items.

Data prepared in: 0.556479s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.00718614

Final training RMSE: 0.084246

In [106]:
graphlab.recommender.util.compare_models(test, [m1_t1,m1_t2,m1_t3,m1], model_names=["m1_t1","m1_t2","m1_t3","m1"],
                                         user_sample=1.0, metric='auto',
                                         target='rating', exclude_known_for_precision_recall=True, 
                                         make_plot=True, verbose=False)

[{'precision_recall_by_user': Columns:
  	userId	int
  	cutoff	int
  	precision	float
  	recall	float
  	count	int
  
  Rows: 12078
  
  Data:
  +--------+--------+-----------+--------+-------+
  | userId | cutoff | precision | recall | count |
  +--------+--------+-----------+--------+-------+
  |   1    |   1    |    0.0    |  0.0   |   1   |
  |   1    |   2    |    0.0    |  0.0   |   1   |
  |   1    |   3    |    0.0    |  0.0   |   1   |
  |   1    |   4    |    0.0    |  0.0   |   1   |
  |   1    |   5    |    0.0    |  0.0   |   1   |
  |   1    |   6    |    0.0    |  0.0   |   1   |
  |   1    |   7    |    0.0    |  0.0   |   1   |
  |   1    |   8    |    0.0    |  0.0   |   1   |
  |   1    |   9    |    0.0    |  0.0   |   1   |
  |   1    |   10   |    0.0    |  0.0   |   1   |
  +--------+--------+-----------+--------+-------+
  [12078 rows x 5 columns]
  Note: Only the head of the SFrame is printed.
  You can use print_rows(num_rows=m, num_columns=n) to print more ro

In [111]:
model_comp1 = graphlab.compare(test, [m1_t1,m1_t2,m1_t3,m1],target='rating')

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+------------------+-------------------+
|   1    | 0.00447093889717 | 0.000156224591503 |
|   2    | 0.00596125186289 | 0.000330269559829 |
|   3    | 0.00645802285147 | 0.000440660336457 |
|   4    | 0.00931445603577 |  0.00107448633492 |
|   5    | 0.0104321907601  |  0.00165845296483 |
|   6    | 0.0101838052658  |  0.00210994116668 |
|   7    | 0.0108579944646  |  0.00260936954202 |
|   8    | 0.0109910581222  |  0.00285313909571 |
|   9    | 0.0112601424077  |  0.00347953670471 |
|   10   | 0.0110283159463  |  0.00361615684896 |
+--------+------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+---------

# Plotting Model Performance Using The Evaluation Results (to-do)

In [112]:
model_comp1.dtype

<bound method SFrame.dtype of Columns:
	dataset	str
	metric	str
	model	str
	model_type	str
	results	dict

Rows: 4

Data:
+---------+------------------+-------+--------------------------+
| dataset |      metric      | model |        model_type        |
+---------+------------------+-------+--------------------------+
|   test  | precision_recall | m1_t1 | FactorizationRecommender |
|   test  | precision_recall | m1_t2 | FactorizationRecommender |
|   test  | precision_recall | m1_t3 | FactorizationRecommender |
|   test  | precision_recall |   m1  | FactorizationRecommender |
+---------+------------------+-------+--------------------------+
+-------------------------------+
|            results            |
+-------------------------------+
| {'precision_recall': [{'cu... |
| {'precision_recall': [{'cu... |
| {'precision_recall': [{'cu... |
| {'precision_recall': [{'cu... |
+-------------------------------+
[4 rows x 5 columns]
>

In [114]:
model_comp1_df = model_comp1.to_dataframe()

In [120]:
model_comp1_df["results"]

0    {u'precision_recall': [{u'cutoff': 1, u'precis...
1    {u'precision_recall': [{u'cutoff': 1, u'precis...
2    {u'precision_recall': [{u'cutoff': 1, u'precis...
3    {u'precision_recall': [{u'cutoff': 1, u'precis...
Name: results, dtype: object

# Cross Validation (Will complete by tonight)

In [ ]:
#K-Fold data split
all_data.to_csv("full_dataset.csv")
hints = [int,int ,int ,float,str, str, str, str, str, str,str,str,str,str,str,str, str,str,float, int, float]
data = graphlab.SFrame.read_csv('full_dataset.csv',column_type_hints= hints)
kfolds = graphlab.cross_validation.KFold(data, 5)
for train, test in kfolds:    
    model = graphlab.factorization_recommender.create(train, target='rating', user_id='userId',
                                               item_id='movieId',solver= "adagrad" , user_data = user_metadata,
                                               item_data = movie_metadata, num_factors=4)
    print(model.evaluate(test))


Finished parsing file C:\Users\hp\Desktop\Jupyter Notebooks\Personalisation-master\Personalisation-master\full_dataset.csv

Parsing completed. Parsed 100004 lines in 0.737376 secs.

Recsys training: model = factorization_recommender

Preparing data set.

Data has 80003 observations with 671 users and 9125 items.

...

Final objective value: 0.0129621

Final training RMSE: 0.11313


Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    |        0.0        |        0.0        |
|   4    |        0.0        |        0.0        |
|   5    |        0.0        |        0.0        |
|   6    |  0.00104166666667 | 2.70562770563e-05 |
|   7    | 0.000892857142857 | 2.70562770563e-05 |
|   8    |     0.00078125    | 2.70562770563e-05 |
|   9    | 0.000694444444444 | 2.70562770563e-05 |
|   10   |      0.000625     | 2.70562770563e-05 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 0.993018906392737)

Per User RMSE (best)
+--------+-------+----------------+
| userId | count |      rmse      |
+--------+-------+----------------+
|  229   |   33  | 0.483189095594 |
+-----

Recsys training: model = factorization_recommender

Preparing data set.